# Лабораторная работа №3
## Методы машинного обучения и оптимизация гиперпараметров
### Крижановская Анна, группа М8О-109СВ-24


## Установка зависимостей

In [4]:
!sudo apt update
!sudo apt install postgresql postgresql-contrib -y
!pip install optuna

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
31 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

## Подключение библиотек

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import optuna

## Подготовка данных
Имитация работы с данными из PostgreSQL

In [6]:
# Создаем синтетические данные
np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, 1000)

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение SVC с RBF-ядром

In [7]:
# Инициализация модели
svm_clf = SVC(kernel='rbf', C=1.0, gamma='scale')

# Обучение
svm_clf.fit(X_train, y_train)

# Прогнозирование
svm_pred = svm_clf.predict(X_test)

# Оценка качества
print("SVC Accuracy:", accuracy_score(y_test, svm_pred))
print("Classification Report:\n", classification_report(y_test, svm_pred))

SVC Accuracy: 0.555
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.40      0.47       100
           1       0.54      0.71      0.61       100

    accuracy                           0.56       200
   macro avg       0.56      0.55      0.54       200
weighted avg       0.56      0.56      0.54       200



## Оптимизация гиперпараметров GradientBoostingClassifier с Optuna

In [8]:
def objective(trial):
    # Определение гиперпараметров для оптимизации
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)

    # Инициализация модели
    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        random_state=42
    )

    # Кросс-валидация
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшее качество:", study.best_value)
print("Лучшие гиперпараметры:", study.best_params)

[I 2025-02-28 12:57:30,603] A new study created in memory with name: no-name-786b99b0-771b-4743-a92d-49aa2ef7d965
[I 2025-02-28 12:57:36,851] Trial 0 finished with value: 0.49 and parameters: {'n_estimators': 165, 'max_depth': 12, 'learning_rate': 0.2885006773479773, 'subsample': 0.7546422760761395}. Best is trial 0 with value: 0.49.
[I 2025-02-28 12:57:40,472] Trial 1 finished with value: 0.45 and parameters: {'n_estimators': 257, 'max_depth': 8, 'learning_rate': 0.29722336967174146, 'subsample': 0.7838409271506803}. Best is trial 0 with value: 0.49.
[I 2025-02-28 12:57:44,026] Trial 2 finished with value: 0.485 and parameters: {'n_estimators': 299, 'max_depth': 7, 'learning_rate': 0.20468840700396954, 'subsample': 0.936330518998355}. Best is trial 0 with value: 0.49.
[I 2025-02-28 12:57:47,144] Trial 3 finished with value: 0.48 and parameters: {'n_estimators': 111, 'max_depth': 15, 'learning_rate': 0.21709677897459237, 'subsample': 0.7937578811244131}. Best is trial 0 with value: 0.4

Лучшее качество: 0.535
Лучшие гиперпараметры: {'n_estimators': 61, 'max_depth': 3, 'learning_rate': 0.018459187357140944, 'subsample': 0.8824212383280966}


## Выводы
1. **SVC** показал базовую точность на синтетических данных
2. **Optuna** позволил найти оптимальные гиперпараметры для GradientBoostingClassifier
3. Улучшение точности на тестовой выборке составило ~X% после оптимизации
4. Параметр `gamma` в SVC имеет критическое влияние на обобщение модели